In [ ]:


!pip install kaggle

# Upload your kaggle.json file (API key)
from google.colab import files
files.upload()

# Move the kaggle.json file to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset (replace with your dataset's name and path)
!kaggle datasets download -d dhananjaya45/new-merged-data-deepfakeaudio

# Unzip the dataset
!unzip new-merged-data-deepfakeaudio.zip -d /content/new_data # replace with your preferred directory


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/dhananjaya45/new-merged-data-deepfakeaudio
License(s): unknown
Archive:  new-merged-data-deepfakeaudio.zip
  inflating: /content/new_data/eval_fake/0adf0a55b12c1880b0db6bcd59403727.wav  
  inflating: /content/new_data/eval_fake/173506125650959p38fcl-voicemaker.in-speech.mp3  
  inflating: /content/new_data/eval_fake/1b8340a8a5d5f4b90f99a7356aaa8020.wav  
  inflating: /content/new_data/eval_fake/1dec403b957925dd446732d3cd22324b.wav  
  inflating: /content/new_data/eval_fake/2a64cf5f4b238aaea8d7ea2dd1eba6d7.wav  
  inflating: /content/new_data/eval_fake/2b25f1de377d75c204a6ce2b295f3a5f.wav  
  inflating: /content/new_data/eval_fake/371414e9cb8a6ab076a77bde0d40aa61.wav  
  inflating: /content/new_data/eval_fake/4a4a624e19ff5115ce7b124614cf2b1e.wav  
  inflating: /content/new_data/eval_fake/6afd8e5eaa9d97e4a4c0a126d228452a.wav  
  inflating: /content/new_data/eval_fake/822fbcd927fcac035d0cacc0df070bb0.wav  
  in

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
model_wav2vec.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [ ]:
# Function to extract features using Wav2Vec2
def extract_embedding(audio_path):
    speech, sr = torchaudio.load(audio_path)
    if speech.shape[0] > 1:
        speech = torch.mean(speech, dim=0, keepdim=True)  # Convert to mono if stereo
    speech = torchaudio.functional.resample(speech, sr, 16000).squeeze(0)
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model_wav2vec(inputs.input_values.to(device)).last_hidden_state
    return outputs.squeeze(0).cpu().numpy()

In [ ]:
# Dataset loading and labeling
def load_dataset(data_dir, include_eval=False):
    features, labels = [], []
    paths = [("real", 1), ("fake", 0)]
    if include_eval:
        paths += [("eval_real", 1), ("eval_fake", 0)]

    for label_name, label_val in paths:
        folder = os.path.join(data_dir, label_name)
        print(f"Processing: {label_name}")
        for file in tqdm(os.listdir(folder)):
            if file.endswith(".wav"):
                try:
                    emb = extract_embedding(os.path.join(folder, file))
                    features.append(emb)
                    labels.append(label_val)
                except Exception as e:
                    print(f"Failed for {file}: {e}")

    return features, np.array(labels)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self, X, y):
        self.X = X  # Keep as list of numpy arrays (no conversion here)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_tensor = torch.tensor(self.X[idx], dtype=torch.float32)  # Convert individually
        return x_tensor, self.y[idx]


# Collate function to pad sequences
def collate_fn(batch):
    X, y = zip(*batch)
    X_padded = nn.utils.rnn.pad_sequence(X, batch_first=True)
    y = torch.tensor(y, dtype=torch.long)
    return X_padded, y

In [ ]:
class AudioDeepfakeDetector(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128):
        super().__init__()
        self.bilstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        lstm_out, _ = self.bilstm(x)
        attn_weights = F.softmax(self.attn(lstm_out), dim=1)
        attn_output = torch.sum(attn_weights * lstm_out, dim=1)
        return self.classifier(attn_output)

In [ ]:
# Training function
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct = 0.0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    return running_loss / len(loader), correct / len(loader.dataset)

# Evaluation function
def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
    return running_loss / len(loader), correct / len(loader.dataset)

# Main
if __name__ == "__main__":
    DATA_PATH = "/content/new_data"

    X, y = load_dataset(DATA_PATH)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    train_dataset = AudioDataset(X_train, y_train)
    val_dataset = AudioDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)

    model = AudioDeepfakeDetector().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_val_acc = 0.0
    os.makedirs("model", exist_ok=True)

    for epoch in range(20):
        train_loss, train_acc = train(model, train_loader, criterion, optimizer)
        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "model/best_model.pth")
            print("\t✔️ New best model saved.")


Processing: real


100%|██████████| 900/900 [00:49<00:00, 18.08it/s]


Processing: fake


100%|██████████| 900/900 [00:26<00:00, 33.73it/s]


Epoch 1 | Train Loss: 0.6779, Acc: 0.5799 | Val Loss: 0.6717, Acc: 0.6042
	✔️ New best model saved.
Epoch 2 | Train Loss: 0.6603, Acc: 0.6321 | Val Loss: 0.6407, Acc: 0.6726
	✔️ New best model saved.
Epoch 3 | Train Loss: 0.6078, Acc: 0.6978 | Val Loss: 0.5762, Acc: 0.7143
	✔️ New best model saved.
Epoch 4 | Train Loss: 0.5583, Acc: 0.7231 | Val Loss: 0.5546, Acc: 0.7173
	✔️ New best model saved.
Epoch 5 | Train Loss: 0.5164, Acc: 0.7485 | Val Loss: 0.5195, Acc: 0.7232
	✔️ New best model saved.
Epoch 6 | Train Loss: 0.4898, Acc: 0.7515 | Val Loss: 0.5088, Acc: 0.7440
	✔️ New best model saved.
Epoch 7 | Train Loss: 0.4737, Acc: 0.7776 | Val Loss: 0.5291, Acc: 0.7381
Epoch 8 | Train Loss: 0.4637, Acc: 0.7873 | Val Loss: 0.5373, Acc: 0.7024
Epoch 9 | Train Loss: 0.4509, Acc: 0.7940 | Val Loss: 0.4955, Acc: 0.7560
	✔️ New best model saved.
Epoch 10 | Train Loss: 0.4498, Acc: 0.7978 | Val Loss: 0.4953, Acc: 0.7470
Epoch 11 | Train Loss: 0.4237, Acc: 0.8127 | Val Loss: 0.5473, Acc: 0.7500
Ep

In [ ]:
model = AudioDeepfakeDetector().to(device)
model.load_state_dict(torch.load("/content/best_model.pth"))
model.eval()

def predict_audio(file_path):
    emb = extract_embedding(file_path)  # shape: [seq_len, 768]
    emb_tensor = torch.tensor(emb, dtype=torch.float32).unsqueeze(0).to(device)  # [1, seq_len, 768]
    with torch.no_grad():
        output = model(emb_tensor)
        pred = torch.argmax(output, dim=1).item()
        return "REAL" if pred == 1 else "FAKE"

predict_audio("/content/new_data/eval_fake/6afd8e5eaa9d97e4a4c0a126d228452a.wav")

'FAKE'

In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

# Make sure this is the same as used during training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = AudioDeepfakeDetector().to(device)
model.load_state_dict(torch.load("/content/best_model.pth", map_location=device))
model.eval()

# Evaluation paths
eval_real_path = "/content/new_data/eval_real"
eval_fake_path = "/content/new_data/eval_fake"

# Store results
y_true = []
y_pred = []

# Prediction Function
def predict_audio(file_path):
    emb = extract_embedding(file_path)
    if emb is None:
        return None  # skip if failed to extract
    emb_tensor = torch.tensor(emb, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(emb_tensor)
        pred = torch.argmax(output, dim=1).item()
    return pred

# Evaluate Real
print("Evaluating Real Files...")
for file in tqdm(os.listdir(eval_real_path)):
    file_path = os.path.join(eval_real_path, file)
    pred = predict_audio(file_path)
    if pred is not None:
        y_true.append(1)  # 1 for real
        y_pred.append(pred)

# Evaluate Fake
print("Evaluating Fake Files...")
for file in tqdm(os.listdir(eval_fake_path)):
    file_path = os.path.join(eval_fake_path, file)
    pred = predict_audio(file_path)
    if pred is not None:
        y_true.append(0)  # 0 for fake
        y_pred.append(pred)

# Final Metrics
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Fake", "Real"]))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("Accuracy Score:")
print(f"{accuracy_score(y_true, y_pred) * 100:.2f}%")


Evaluating Real Files...


100%|██████████| 75/75 [00:06<00:00, 11.83it/s]


Evaluating Fake Files...


100%|██████████| 75/75 [00:03<00:00, 22.32it/s]



Classification Report:
              precision    recall  f1-score   support

        Fake       0.93      0.57      0.71        75
        Real       0.69      0.96      0.80        75

    accuracy                           0.77       150
   macro avg       0.81      0.77      0.76       150
weighted avg       0.81      0.77      0.76       150

Confusion Matrix:
[[43 32]
 [ 3 72]]
Accuracy Score:
76.67%
